# **Simulación del Modelo de Ising con la dinámica de Kawasaki**
## Problema Voluntario 2
### Física Computacional - 2025

---

**Irene Blanco Ramos**

**Fecha:** 11 de Junio de 2025

---


# Índice

1. [Introducción](#introducción)
2. [Simulación del Modelo de Ising](#simulación-del-modelo-de-ising)
3. [Animaciones](#animaciones)
4. [Curva de magnetización](#curva-de-magnetización)
5. [Densidad de partículas media en la dirección Y](#densidad-de-partículas-media-en-la-dirección-y)
6. [Energía media por partícula](#energía-media-por-partícula)
7. [Calor espefífico](#calor-espefífico)
8. [Susceptibilidad magnética](#susceptibilidad-magnética)
9. [Optimización](#optimización)
10. [Conclusiones](#conclusiones)
11. [Referencias](#referencias)
---

# Introducción
Este informe presenta un estudio del Modelo de Ising bidimensional, implementado a través de una simulación de Monte Carlo  en C que utiliza la dinámica de Kawasaki. El objetivo principal de este trabajo es caracterizar el comportamiento y las transiciones de fase de los materiales ferromagnéticos. 

Los materiales ferromagnéticos son sustancias que presentan dipolos magnéticos permanentes. En los átomos de estos materiales, los momentos magnéticos o espines se distribuyen aleatoriamente en el metal, aunque suelen pueden estar alineados con sus vecinos, formando dominios magnéticos (cada dominio con una orientación diferente). Cuando una fracción considerable de dominios se alinean, el material adquiere una magentización neta no nula. Esto ocurre, por ejemplo, cuando el material se encuentra bajo la influencia de un campo magnético externo y los espines se alinean con éste.

La alineación de los espines es un estado que se mantiene únicamente a bajas temperaturas. Por debajo de una temperatura crítica, conocida como temperatura de Curie ($T_C$), los espines conservan su orientación colectiva. Sin embargo, al superar esta temperatura, los espines se orientan de manera aleatoria, resultando en una ausencia de magnetización neta. La temperatura de Curie es un punto de transición de fase en el que el material pasa de ferromagnético a paramagnético, donde el material paramagnético se caracteriza por su magnetización neta nula.

La transición de fase ocurre porque en el sistema compiten dos factores: la energía y la temperatura. Teniendo solamente en cuenta la minimización de la energía, el sistema tendería siempre a ordenarse. Sin embargo, cuando aumenta la temperatura los espines adquieren mayor facilidad para moverse y orientarse libremente, llevando al sistema a una fase desordenada. Por tanto, el estado del sistema va a depender de si predomina la agitación térmica (desorden) o no (tendencia al mínimo energético, orden).


El Modelo de Ising pretende estudiar este comportamiendo de forma sencilla. Se considera un sólido ferromagnético como una malla cuadrada, donde en cada punto se sitúa un momento magnético de +1 o -1. A partir de la energía de interacción de los espines, variando los distintos parámetros de la simulación, se han analizado las propiedades termodinámicas más relevantes en función de la temperatura: magnetización, energía media, calor específico y susceptibilidad magnética.

----

# Simulación del Modelo de Ising
Como ya se ha mencionado, el Modelo de Ising consisten en una red bidimensional cuadrada de tamaño $N$ y de direcciones $x$ e $y$ con un espín en cada punto de la red que puede tomar el valor de -1 o +1. Aplicando la dinámica de Kawasaki, se seleccionan aleatoriamente dos espines vecinos de la red que tengan valores opuestos para intentar intercambiarlos. Este intercambio solo se realiza si cumple con el criterio de Metrópolis, es decir, si disminuye la energía del sistema o, si la energía aumenta, con una cierta probabilidad dependiente de la temperatura (algoritmo de aceptación-rechazo).

En la red se han impuesto condiciones de contorno periódicas en la dirección horizontal ($y$). Esto significa que los extremos izquierdo y derecho de la red están conectados entre sí: el primer y el último elemento de cada fila son vecinos. Así, un espín en el borde derecho tiene como vecino a uno en el borde izquierdo de la misma fila, y viceversa.
En la dirección vertical ($x$), las condiciones son libres: los espines de la primera y última fila no tienen vecinos fuera de la red, y además sus valores están fijados. 

Además, las condiciones iniciales de la red se establecen para que la magnetización total sea nula y la configuración esté desordenada. La primera fila de la red se fija con todos los espines en -1 y la última fila con todos los espines en +1.
Las filas intermedias se rellenan con la mitad de los espines en +1 y la otra mitad en -1, distribuidos de forma aleatoria en cada fila (usando un barajado tipo Fisher-Yates).
De este modo, al comenzar la simulación, hay el mismo número de espines +1 que -1, pero su posición es aleatoria, lo que garantiza una magnetización inicial igual a cero y un estado desordenado. También se ha hecho uso de una configuración inicial ordenada, con la mitad superior de la red a +1 y la inferior a -1, en algunos casos concretos.

### Energía total de la Red
Para el valor de un espín en la posición $(i,j)$, $s_{i,j}$, la energía de una configuración de espines está dada por la ecuación:

$$E(C) = -\frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} s_{i,j}(s_{i+1,j} + s_{i-1,j} + s_{i,j+1} + s_{i,j-1})$$

Para que la simulación sea eficiente, la energía total de la red no se calcula en cada paso del bucle Monte carlo principal. En su lugar, se determina una única vez al principio de la simulación.

Durante la simulación, cuando se intenta un "intercambio Kawasaki" (donde dos espines vecinos intentan cambiar de posición), solo la energía de los espines directamente implicados y sus alrededores cambia. Por eso, en cada paso, la energía total de la red se actualiza sumando la diferencia de energía ($\Delta E$) que resulta de ese intercambio específico. Esto evita tener que volver a calcular toda la energía de la red, reduciendo el coste computacional.

La diferencia de energía ($\Delta E$) para un intercambio Kawasaki se calcula eligiendo dos espines vecinos que tienen orientaciones opuestas. Se calcula la energía que esos dos espines tienen con sus vecinos antes de que cambien de lugar. Esto se hace usando una función que suma las interacciones de cada espín con sus propios vecinos (sin incluir el otro espín del par que se va a mover).
Los dos espines intercambian sus posiciones en la red y se vuelve a calcular la energía que esos dos espines tienen con sus vecinos después de que hayan cambiado de lugar.


La diferencia de energía se obtiene restando la energía "antes" de la energía "después" del intercambio.
Si $\Delta E > 0$ (la energía aumenta), el cambio se acepta con una probabilidad $p = \exp(-\Delta E / T)$, donde $T$ es la temperatura (se ha tomado la constante de Boltzmann $k_B=1$).
Si el sistema decide no aceptar el intercambio, los espines vuelven a sus posiciones originales y la red regresa al estado anterior.

Este método es efectivo porque se enfoca en los pequeños cambios locales de energía, sin necesidad de recalcular toda la red, lo que hace la simulación mucho más rápida.

### Temperatura crítica esperada

La solución exacta del modelo de Ising es conocida y fue resuelta analíticamente por Onsager para el caso de dos dimensiones. Encontró que la temperatura crítica para la transición de fase en el límite termodinámico, cuando $N \to \infty$ y $V \to \infty$, es:
$$
T=\frac{2}{ln(1+\sqrt{2})} \approx 2.269
$$

Es complicado que determinemos de forma precisa esta temperatura crítica porque estudiamos sistemas de $N$ finitos. A pesar de esto, es posible evaluar las magnitudes a estudiar para distintos tamaños de $N$ prestando atención al comportamiento del sistema en los puntos cercanos a $T\approx2$.


---

# Animaciones
A continuación se adjuntan varias animaciones de la simulación para dos temperaturas diferentes con configuraciones iniciales desordenadas y ordenadas. Los puntos en negro representan el espín con valor negativo y los blancos con valor positivo.

**Animaciones para $T=1.0$ con configuración ordenada (derecha) y aleatoria (izquierda)**

<div style="display: flex; justify-content: center; gap: 20px;">
  <video width="400" controls>
    <source src="ising1.mp4" type="video/mp4">
    Tu navegador no soporta la etiqueta de video.
  </video>
  <video width="400" controls>
    <source src="ising2.mp4" type="video/mp4">
    Tu navegador no soporta la etiqueta de video.
  </video>
</div>

**Animaciones para $T=1.0$ con configuración ordenada (derecha) y aleatoria (izquierda)**
<div style="display: flex; justify-content: center; gap: 20px;">
  <video width="400" controls>
    <source src="ising4.mp4" type="video/mp4">
    Tu navegador no soporta la etiqueta de video.
  </video>
  <video width="400" controls>
    <source src="ising3.mp4" type="video/mp4">
    Tu navegador no soporta la etiqueta de video.
  </video>
</div>

Para la temperatura más baja $T=1.0$ se ve como para la configuración aleatoria los espines se organizan en dominios y para la configuración ordenada los espines apenas cambian porque la configuración ya es ordenada (comportamiento ferromagnético).

Para la temperatura más alta $T=4.0$ se observa que la configuración aleatoria inicial no se ordena mientras que la configuración ordenada tiende a desordenarse (comportamiento paramagnético).

De aquí se puede deducir que la temperatura crítica de transición de fase se encuentra entre 1.0 y 4.0.

Esto también se aprecia si representamos la energía en función de los pasos Monte Carlo. Para conficuración aleatoria, la energía converge a un valor para bajas temperaturas pero no para altas temperaturas.

**Estudio de la convergencia de la energía media para $T=1.0$ y $T=0.4$ con configuración inicial aleatoria**

<p align="center">
  <img src="evolucion energia.png" alt="evolucion energia" width="400"/>
  <img src="evolucion energia 4.png" alt="evolucion energia 4" width="400"/>
</p>

Para $T=1.0$ la energía converge a -24000 aproximadamente pero para $T=4.0$ los valores de la energía oscilan sin converger, lo que prueba que el sistema no se estabiliza y continúa desordenado.

---

# Curva de magnetización

Como se ha impuesto que la magnetización neta de la red fuese nula, la magnetización se calcula por separado para la mitad superior y la mitad inferior de la red. Para cada mitad, se suman los valores de los espines (+1 o -1) en todas las filas correspondientes (de 0 a $N/2-1$ para la superior, de $N/2$ a $N-1$ para la inferior).
Se obtiene el valor absoluto de la suma y se divide entre el número total de espines en esa mitad $(N/2) N$, obteniendo así la magnetización promedio por espín en cada región.
Esto se realiza en las funciones **magnetizacion_mitad_superior** y **magnetizacion_mitad_inferior**.
Durante la simulación, estos valores se almacenan cada 100 pasos Monte Carlo para analizar la evolución y calcular promedios y desviaciones estándar.

Esta gráfica, muestra la magnetización en función de la temperatura para tres tamaños de la red diferentes.

<p align="center">
  <img src="magnetizacion.png" alt="magnetizacion" width="700">
</p>

En la figura, se observa una caída brusca de la magnetización al aumentar la temperatura, lo que indica la transición de fase de ferromagnético a paramagnético. 
Para redes más grandes ($N=128$), la transición es más abrupta y la magnetización inicial es mayor, lo que es consistente con el comportamiento esperado en el límite termodinámico. 


La caída brusca también se acentuaba al aumentar el número de pasos Monte Carlo. 
Por ello, se han considerado un elevado número de pasos (100000) para intentar apreciar mejor la temperatura crítica. La magnetización disminuye rápidamente alrededor de $T \approx 2$, valor próximo a la temperatura crítica esperada. Para $T > 3$, la magnetización se mantiene cerca de cero para todos los tamaños de red, indicando un estado desordenado (paramagnético).

En la resolución analítica del Modelo de Ising se llega a que la magnetización entorno al punto crítico viene dada por:
$$
M(T)  \sim A (T_c - T)^{1/8}
$$
donde $A$ es una constante y $\beta=1/8$ es el exponente crítico de la magnetización. 

En general, los exponentes críticos tratan de describir el comportamiento de las funciones termodinámicas cuando el sistema se aproxima al punto de cambio de fase. En el caso de la magnetización:
$$M(T) = B(T_c - T)^{\beta}$$
Aplicando logaritmos y renombrando a la constante $B$:

$$ ln[M(T)] = \beta ln(T_c - T) + B $$

Haciendo un ajuste de esta expresión, es posible calcula el exponente crítico y la temperatura $T_C$. El ajuste se ha realizado para la red de 128 por ser la de mayor $N$ seleccionando los valores de la magentización cercanos al punto crítico.
<p align="center">
  <img src="exponente critico.png" alt="exponentecritico" width="500">
</p>

El ajuste devuelve como parámetros $\beta= 0.4592$ y $T_C=2.5630$. La temperatura crítica se aproxima a la predicha por Onsager pero el exponente crítico difiere de 0.125.


---

# Densidad de partículas media en la dirección Y

En la simulación, la densidad media por partícula en la dirección $y$ se calcula para una fila concreta usando la función **densidad_media_y**.
Esta función recorre todos los espines de la fila seleccionada y cuenta cuántos tienen valor +1. Luego, divide ese número entre el total de espines de la fila ($N$). La densidad media es el porcentaje de espines positivos (+1) en esa fila, y representa la fracción de partículas con espín +1 en la dirección y para la fila seleccionada. La fila seleccionada ha sido la fila central de la red de índice $N/2$. Esto permite analizar la distribución de espines en una zona representativa del sistema, lejos de los bordes donde los espines están fijados.

<p align="center">
  <img src="densidad media.png" alt="densidad media" width="700">
</p>

En este apartado, los datos se han tomado con una configuración inicial ordenada, por eso para el tamaño de red más grande (N=128), la densidad media comienza en 1 para bajas temperaturas, indicando que la fila central está completamente ocupada por espines +1 al inicio. 

Al aumentar la temperatura, la densidad media disminuye rápidamente y se estabiliza alrededor de 0.5 para todos los tamaños de red, lo que prueba una distribución aleatoria de espines (+1 y -1) a altas temperaturas. Además, los valores de densidad media para diferentes tamaños de red convergen a altas temperaturas, mostrando que el sistema se comporta de manera similar independientemente del tamaño cuando está desordenado.


---

# Energía media por partícula

Para calcular la energía media por partícula, primero se utiliza la función **energia_total_vecinos**, que suma las interacciones de cada espín con sus vecinos.
La energía total se calcula solo una vez al inicio y luego se actualiza en cada paso Kawasaki sumando el cambio de energía local. Se realiza un promedio cada 100 pasos Monte Carlo, almacenando el valor de la energía total. Al final de la simulación, se calcula la media de todas las energías almacenadas y se normaliza dividiendo la energía media entre el doble del número total de espines. 

Se divide entre el doble del número total de espines ($2N^2$) porque, al calcular la energía total de la red, cada par de espines vecinos se cuenta dos veces (una vez por cada espín del par). De esta forma, se evita este doble conteo y se obtiene un valor correcto de la energía media normalizada.

Esto da como resultado, para distintos tamaños, la siguiente gráfica:

<p align="center">
  <img src="energia media.png" alt="energia media" width="700">
</p>

Para temperaturas bajas, la energía media por partícula es cercana a -1. El sistema está en un estado ordenado y de mínima energía (ferromagnético). 

Al aumentar la temperatura, la energía media por partícula sube progresivamente, reflejando el aumento del desorden en el sistema. Se observa un cambio rápido en la pendiente de la curva alrededor de $T \approx 2$, que corresponde al punto de transición de fase.

Además, las curvas para diferentes tamaños de red (N=32, 64, 128) prácticamente coinciden, mostrando que el comportamiento es similar independientemente del tamaño de la red.

---

# Calor espefífico

El calor específico se ha calculado partiendo de las fluctuación de la energía, utilizando la expresión:

$$c_N = \frac{1}{N^2T^2} [\langle E^2 \rangle - \langle E \rangle^2]$$

Para distintos tamaños de la red, el calor específico muestra la siguiente relación con la temperatura:
<p align="center">
  <img src="calor especifico.png" alt="calor especifico" width="700">
</p>

La gráfica revela un pico pronunciado que se va desplazando hacia la derecha a medida que aumentamos el tamaño de la red ocupando un rango de temperaturas entre $[2,3]$. Este máximo tan marcado señala la transición de fase del sistema. Es importante notar cómo el valor máximo del calor específico aumenta y el pico se estrecha también conforme se incrementa el tamaño de la red $(N)$.

De acuerdo con la solución de Onsager, la capacidad calorífica presenta una singularidad logarítmica en la temperatura crítica:
$$C(T) \sim -\ln \left|1 - \frac{T}{T_c}\right|$$
Esto para redes finitas no sucede. En el artículo "The Analytical Expressions for a Finite-Size 2D Ising Model" [3], se comprueba experimentalmente que el pico de la capacidad calorífica $C$ crece de forma logarítmica con el tamaño de la red y que se va desplazando ligeramente hacia la derecha de la temperatura crítica de Onsager. La ecuaciones experimentales que obtienen en el artículo relacionando $C$ en el punto crítico con $N$ son las siguientes:
$$\beta_c = \beta_{ONS} \left(1 + \frac{5}{4\sqrt{N}}\right)$$
$$C(N) = \frac{4\beta_c^2}{\pi} (\ln N - 1.7808)$$

donde $\beta_{ONS}$ es el valor de $\beta$ en la temperatura crítica de Onsager y $\beta_c$ en la tempetarura crítica experimental.

La primera expresión refleja ese desplazamiento hacia la derecha del pico de la capacidad específica al  aumentar $N$. Esto coincide con los resultados obtenidos en la simulación. La segunda expresión expone que el crecimiento del valor pico de la capacidad calorífica aumenta con el logaritmo de $N$ $(C \sim ln(N))$. No obstante, en la simulación los máximos no siguen una tendencia logarítmica con $N$ si no que crecen más rápidamente. Además, en la simulación representamos el calor específico no la capacidad calorífica, esta propiedad es intensiva que no debería depender de $N$.

En la siguiente tabla se comparan los valores de las temperaturas críticas calculadas con la ecuación del artículo y las obtenidas en la simulación:
<div align="center">

| $N$ | $C_v$ Simulación  | $T_C$ Simulación  | $T_C$ Artículo  |
|-----|--------|-------------------|----------------|
| 32  | 2.52 | 2.0 | 1.86   |
| 80  | 38.28 | 2.3 | 1.99   |
| 128 | 104.59 | 2.5 | 2.04    |

</div>

Vemos que las temperaturas de la simulación superan a las calculadas con la expresión experimental.

---

# Susceptibilidad magnética

La susceptibilidad magnética se ha hallado a partir de las fluctuaciones de la magnetización en cada dominio, según la ecuación:

$$\chi_N = \frac{1}{N^2T} [\langle M^2 \rangle - \langle M \rangle^2]$$

Esta figura muestra la tendencia seguida por la susceptibilidad al variar la temperatura:
<p align="center">
  <img src="susceptibilidad magnetica.png" alt="susceptibilidad" width="700">
</p>

Para cada tamaño de red, la susceptibilidad magnética presenta un máximo alrededor de $T \approx 2$, lo que se corresponde con la presencia de la transición de fase. El valor máximo de la susceptibilidad disminuye y se hace más ancho al aumentar el tamaño de la red. Para redes más grandes, el pico es menos pronunciado, pero sigue indicando la transición.
Para temperaturas altas, la susceptibilidad magnética tiende a valores bajos y similares para todos los tamaños de red, lo que caracteriza a un estado desordenado (paramagnético).

----

## Cálculo de errores

Se han añadido barras de error en las gráficas de las distintas magnitudes representadas.
Los errores en las medidas se han calculado utilizando la desviación estándar de la media. Para un conjunto de datos independientes y con la misma distribución de media finita y varianza finita, el intervalo de confianza para el valor real se estima como:

- $\left[ \bar{X} - \frac{\sigma}{\sqrt{N}}, \bar{X} + \frac{\sigma}{\sqrt{N}} \right]$ con un 68,26% de probabilidad (1 sigma)
- $\left[ \bar{X} - 2\frac{\sigma}{\sqrt{N}}, \bar{X} + 2\frac{\sigma}{\sqrt{N}} \right]$ con un 95,44% de probabilidad (2 sigmas)
- $\left[ \bar{X} - 3\frac{\sigma}{\sqrt{N}}, \bar{X} + 3\frac{\sigma}{\sqrt{N}} \right]$ con un 99,74% de probabilidad (3 sigmas) 

donde $\bar{X}$ es la media de las medidas, $\sigma$ es la desviación estándar y $N$ es el número de medidas.
Los errores mostrados en las gráficas corresponden al error estándar de la media, que indica el rango donde probablemente se encuentra el valor real con un intervalo de confianza de 3 sigmas.

---

# Optimización

A la hora de elaborar el código de la simulación, inicialmente no se añadió un break en la función **paso_kawasaki**, encargada de intercambiar la energía de los espines vecinos. Al no haber break en el bucle que busca un vecino distinto para el intercambio en la función, el programa seguía comprobando todos los vecinos aunque ya hubiese encontrado uno válido.

El break sirve para salir inmediatamente del bucle en cuanto se encuentra un vecino válido para el intercambio de espines (es decir, un vecino que sea diferente al espín seleccionado). Así, el código no sigue buscando más vecinos una vez que ya ha encontrado uno adecuado, lo que mejora la eficiencia y evita realizar comprobaciones innecesarias.

Representando el tiempo de ejecución frente al tamaño de la red con y sin el break, vemos como ha mejorado la eficiencia y el coste computacional se reduce pasando de una función cuadrática a otra lineal.
<p align="center">
  <img src="break.png" alt="break" width="700">
</p>

Para medir el tiempo de ejecución se ha empleado la función time() de la biblioteca estádar de C (<time.h>). Devuelve el tiempo transcurrido desde un punto de referencia conocido como la "Época" (Epoch). En la mayoría de los sistemas, especialmente los basados en Unix, esta Época se define como 00:00:00 UTC (tiempo universal coordinado) del 1 de enero de 1970. El valor se expresa en segundos. La función devuelve una variable de tipo time_t en segundos. Restando dos valores de time_t, se ha calculado el tiempo de ejecución del bucle principal del programa. 

Por otro lado, otra mejora que se podría haber implementado es el generador Philox de números pseudoaleatorios  de C++ en lugar de rand() de C. Este generador es más sofisticado y de mayor nivel que el estándar por su capacidad de generación paralela de números aleatorios en entornos con muchos hilos o núcleos, como el ordenador Joel. Además, está recomendado en la documentación para ser usado en simulaciones Monte Carlo. Sin embargo, hemos tenido problemas a la hora de implementar este generador en el código en C++.

También cabe destacar que para hacer la simulación tanto de este problema como del Voluntario 3 se ha empleado la IA GitHub Copilot integrada en VSCode. Ha sido de gran ayuda para buscar información sobre funciones básicas de C y Python y generar código para la representación gráfica de los datos.

Por último, ya con el break incluido, se ha intentado optimizar el programa de la simulación utilizando las opciones de optimización -O1, -O2, -O3 con el compilador **AMD Clang** de **Joel**. 

Sin ninguna opción de optimización, el objetivo del compilador es reducir el coste de compilación y hacer que la depuración produzca los resultados esperados. Activar los indicadores de optimización hace que el compilador intente mejorar el rendimiento y/o el tamaño del código a costa del tiempo de compilación y de la capacidad de depurar el programa.

Con -O1, el compilador intenta reducir el tamaño del código y el tiempo de ejecución, sin realizar ninguna optimización que lleve mucho tiempo de compilación. -O2 es un nivel más avanzado que incorpora un conjunto más amplio y potente de optimizaciones. Esta opción aumenta tanto el tiempo de compilación como el rendimiento del código generado. -O3  incluye todas las optimizaciones de -O2 y activa las optimizaciones más agresivas y que pueden consumir un tiempo de compilación mayor. Su prioridad es la máxima velocidad de ejecución del código lo que puede aumentar el tamaño del ejecutable.

Primero, es necesario especificar las características del superordenador Joel y del PC utilizado antes de compararlos. 
| Recurso              | Joel                 | PC                    |
| :------------------- | :--------------------------------------- | :------------------------------------------- |
| **Sistema Operativo**| Ubuntu 22.04.5 LTS                       | Microsoft Windows 11                     |
| **Modelo de la CPU** | AMD EPYC 7401P 24-Core Processor         | 11th Gen Intel(R) Core(TM) i5-1135G7         |
| **CPUs disponibles / Núcleos** | 48 núcleos (físicos/lógicos combinados) | 4 procesadores principales, 8 procesadores lógicos |
| **Memoria RAM empleada**| 2,9 GiB                                  | 7.77 GB (memoria usable)|
| **Memoria RAM total**| 62 GiB                                   | 8.00 GB|
| **Frecuencia de reloj (máx)** | 2000 MHz                             | 2419 Mhz (Frecuencia base)                   |
| **Frecuencia de reloj (mín)** | 1200 MHz                             | *No especificado* |


En la imagen siguiente que ha representado el coste computacional en función del tamaño de la red tanto para el PC como para las opciones de optimización.
<p align="center">
  <img src="optimizacion.png" alt="optimizacion" width="700">
</p>

El PC es más lento que cualquiera de las configuraciones de Joel para todos los tamaños de red presentados. Para $N=128$, el PC toma aproximadamente 205 segundos, mientras que el Joel+O3 toma alrededor de 180 segundos. 

 La línea verde (Joel+O2) es consistentemente más baja (más rápida) que la línea naranja (Joel+O1). Esto confirma que las optimizaciones adicionales que -O2 habilita son beneficiosas para el rendimiento de esta simulación.

A diferencia de lo que se podría esperar, Joel+O3 (línea roja) es consistentemente más lento que Joel+O2 (línea verde) para la mayoría de los tamaños de red, especialmente a medida que aumenta N. El nivel -O3 puede llevar a un rendimiento inferior en ciertos casos debido a efectos de sobre-optimización, por lo tanto la configuración más acertada es -O2.


---

# Conclusiones
Este informe ha caracterizado la transición de fase en el Modelo de Ising bidimensional utilizando la dinámica de Kawasaki. Las simulaciones confirmaron que la temperatura determina el ordenamiento del sistema: a bajas temperaturas, se forman dominios ordenados, mientras que a altas temperaturas, el sistema permanece desordenado, lo que se percibe también en la evolución de la energía. La magnetización, energía media, calor específico y susceptibilidad magnética mostraron las tendencias esperadas, con picos y caídas que coinciden con la temperatura crítica, bastante próxima a la calculada por Onsager. Las propiedades del calor específico, como el aumento del pico con el tamaño de la red y su ligero desplazamiento, también se observaron en la simulación.

En cuanto a la eficiencia computacional, se implementaron mejoras significativas. La adición de un break en la función **paso_kawasaki** optimizó el tiempo de ejecución de una dependencia cuadrática a lineal con el tamaño de la red. Finalmente, las pruebas de optimización del compilador en el sistema  Joel de alto rendimiento revelaron que la configuración -O2 fue la más eficiente para esta simulación, incluso superando a la más agresiva -O3.


---


# Referencias

1. [ Modelo de Ising exponentes críticos](https://www.famaf.unc.edu.ar/~cannas/Notas_TermoII/Notas-TermoII-2010-12.pdf)
2. [Modelo de Ising y temperatura de Curie](https://valbuena.fis.ucm.es/expint/html/fises/ising/ising.html)
3. [The Analytical Expressions for a Finite-Size 2D Ising Model](https://arxiv.org/pdf/1706.02541)
4. [Función Time](https://en.cppreference.com/w/c/chrono/time)
5. [Optimization flags](https://gcc.gnu.org/onlinedocs/gcc/Optimize-Options.html)